# Capstone : Webscrapping using BeautifulSoup

This notebook contains guidances & tasks on the data processing for the application

Hi, my name is Fadhil and I'm currently working on project held by Algorit.ma Data Science School to do Webscrapping using BeautifulSoup. In this Project, I'm trying to collecting datas from www.exchange-rates.org/history/IDR/USD/T and see what can I get from there,then visualizing the datas. Beside that, I'm also using flask dashboard to show up the datas and the visualization.

## *Step 1 : Requesting the Data and Create BeautifulSoup*

Let's begin with requesting the web from the site with `get` method.

In [1]:
import requests
url_get = requests.get('https://www.exchange-rates.org/history/IDR/USD/T')

Visualize what I get from `requests.get` using `.content`

In [2]:
#If I left it blank, it will show up the whole html text. This will only show me what exactly I get from request.get
url_get.content[1:1000] 

b'\n<!DOCTYPE html>\r\n<!--[if lt IE 9]>\r\n    <html class="no-js ie8 oldie" lang=\'en\' xml:lang=\'en\'>\r\n<![endif]-->\r\n<!--[if gt IE 8]><!--> <html class="no-js" lang=\'en\' xml:lang=\'en\'> <!--<![endif]-->\r\n<head><title>Indonesian Rupiah Exchange Rate - US Dollar - Historical Exchange Rates</title>\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta content="C#" name="CODE_LANGUAGE" />\n<meta content="JavaScript" name="vs_defaultClientScript" />\n<meta content="http://schemas.microsoft.com/intellisense/ie5" name="vs_targetSchema" />\n<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=5">\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n\n<meta name="mapping" content="AP" />\n<base href="https://www.exchange-rates.org/" />\n<link rel="dns-prefetch" href="https://ajax.googleapis.com/" />\r\n<link rel="dns-prefetch" href="https://www.google-analytics.com/" />\r\n<link rel="preload" href="https://dol8qrceshfrq.cloud

Since we get unstructured html, we need BeautifulSoup to transform complex html into a complex tree of Python object

In [3]:
from bs4 import BeautifulSoup 
soup = BeautifulSoup(url_get.content,"html.parser")
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [4]:
soup


<!DOCTYPE html>

<!--[if lt IE 9]>
    <html class="no-js ie8 oldie" lang='en' xml:lang='en'>
<![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en" xml:lang="en"> <!--<![endif]-->
<head><title>Indonesian Rupiah Exchange Rate - US Dollar - Historical Exchange Rates</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="C#" name="CODE_LANGUAGE">
<meta content="JavaScript" name="vs_defaultClientScript"/>
<meta content="http://schemas.microsoft.com/intellisense/ie5" name="vs_targetSchema"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<meta content="AP" name="mapping"/>
<base href="https://www.exchange-rates.org/"/>
<link href="https://ajax.googleapis.com/" rel="dns-prefetch"/>
<link href="https://www.google-analytics.com/" rel="dns-prefetch"/>
<link as="image" href="https://dol8qrceshfrq.cloudfront.net/cache/2020-03-18-01-05-58/img/bgr-pattern.p

In [5]:
#it gives the visual representation of the parse tree created from the raw HTML content.
print(soup.prettify()[:1000])

<!DOCTYPE html>
<!--[if lt IE 9]>
    <html class="no-js ie8 oldie" lang='en' xml:lang='en'>
<![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en" xml:lang="en">
 <!--<![endif]-->
 <head>
  <title>
   Indonesian Rupiah Exchange Rate - US Dollar - Historical Exchange Rates
  </title>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="C#" name="CODE_LANGUAGE">
   <meta content="JavaScript" name="vs_defaultClientScript"/>
   <meta content="http://schemas.microsoft.com/intellisense/ie5" name="vs_targetSchema"/>
   <meta content="width=device-width, initial-scale=1, maximum-scale=5" name="viewport"/>
   <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
    <meta content="AP" name="mapping"/>
    <base href="https://www.exchange-rates.org/"/>
    <link href="https://ajax.googleapis.com/" rel="dns-prefetch"/>
    <link href="https://www.google-analytics.com/" rel="dns-prefetch"/>
    <link as="image" href="https://dol8qrceshfrq.cloudfront


## *Step 2 : Finding the right key to scrap the data & Extracting the right information*

In my project, I'm scrapping a webpage that consist a table with Indonesia IDR to US Dollar rate based on date. So then, the right key is `table` ( *'class' : 'table table-striped table-hover table-hover-solid-row table-simple history-data'* )

Since the key is table, I'm going to use .find() to see the html table code that I want to scrap.

In [6]:
table = soup.find('table', attrs={'class':'table table-striped table-hover table-hover-solid-row table-simple history-data'})
print(table.prettify()[1:500])

table class="table table-striped table-hover table-hover-solid-row table-simple history-data">
 <tbody>
  <tr>
   <td>
    11/27/2020
   </td>
   <td>
    Friday
   </td>
   <td>
    <a href="/Rate/USD/IDR/11-27-2020">
     14,078.12448
    </a>
    IDR
   </td>
   <td class="text-narrow-screen-hidden">
    <a href="/Rate/USD/IDR/11-27-2020">
     USD IDR rate for 11/27/2020
    </a>
   </td>
  </tr>
  <tr>
   <td>
    11/26/2020
   </td>
   <td>
    Thursday
   </td>
   <td>
    <a href="/Rate


Based on html code :
+ th stand for table header/header cell 
+ tr stand for table row 
+ td stand for standard cell 

I want to create a table that consist of Date and Rate (IDR to USD), so that I need to take `td[0]` and `td[2]` by using a loop

Here what the looping do to scrap the information:

+ First we need to establish a placeholder to receive the information that we scrap.
+ We named it temp and it's a tuple.
+ Then we will make a loop from one until the the length of the table row,
+ which we will find all cell of the column one and two which contain period and the inflation rate.
+ Then we will append it to our tuple that we prepared before,
+ every one iteration we will scrap one line of the table.

In [7]:
tr = table.find_all('tr')
tr[:2]

[<tr><td>11/27/2020</td><td>Friday</td><td><a href="/Rate/USD/IDR/11-27-2020">14,078.12448</a> IDR</td><td class="text-narrow-screen-hidden"><a href="/Rate/USD/IDR/11-27-2020">USD IDR rate for 11/27/2020</a></td></tr>,
 <tr><td>11/26/2020</td><td>Thursday</td><td><a href="/Rate/USD/IDR/11-26-2020">14,090.70769</a> IDR</td><td class="text-narrow-screen-hidden"><a href="/Rate/USD/IDR/11-26-2020">USD IDR rate for 11/26/2020</a></td></tr>]

In [8]:
#initiating a tuple
temp = []

for i in range(1, len(tr)):
    row = tr[i]
    
    #get tanggal
    Date = row.find_all('td')[0].text
    Date = Date.strip()
    #get rate
    Rate = row.find_all('td')[2].text
    Rate = Rate.strip()
    
    temp.append((Date,Rate))
    #scrapping process
    
temp 

[('11/26/2020', '14,090.70769 IDR'),
 ('11/25/2020', '14,136.09929 IDR'),
 ('11/24/2020', '14,137.48161 IDR'),
 ('11/23/2020', '14,185.57119 IDR'),
 ('11/20/2020', '14,162.60519 IDR'),
 ('11/19/2020', '14,165.50144 IDR'),
 ('11/18/2020', '14,051.29503 IDR'),
 ('11/17/2020', '14,040.16051 IDR'),
 ('11/16/2020', '14,085.69501 IDR'),
 ('11/13/2020', '14,142.77143 IDR'),
 ('11/12/2020', '14,141.24303 IDR'),
 ('11/11/2020', '14,073.46024 IDR'),
 ('11/10/2020', '14,053.76135 IDR'),
 ('11/9/2020', '14,283.29667 IDR'),
 ('11/6/2020', '14,214.49744 IDR'),
 ('11/5/2020', '14,389.14027 IDR'),
 ('11/4/2020', '14,544.44634 IDR'),
 ('11/3/2020', '14,589.01774 IDR'),
 ('11/2/2020', '14,612.89990 IDR'),
 ('10/30/2020', '14,629.16216 IDR'),
 ('10/29/2020', '14,701.62361 IDR'),
 ('10/28/2020', '14,701.92586 IDR'),
 ('10/27/2020', '14,649.78960 IDR'),
 ('10/26/2020', '14,652.88010 IDR'),
 ('10/23/2020', '14,655.04157 IDR'),
 ('10/22/2020', '14,647.90877 IDR'),
 ('10/21/2020', '14,641.61060 IDR'),
 ('10/2

In [9]:
#Sort from the earliest
temp = temp[::-1]
temp

[('5/31/2020', '14,664.14584 IDR'),
 ('6/1/2020', '14,586.73900 IDR'),
 ('6/2/2020', '14,382.27395 IDR'),
 ('6/3/2020', '14,060.78946 IDR'),
 ('6/4/2020', '14,113.82802 IDR'),
 ('6/5/2020', '14,305.96407 IDR'),
 ('6/7/2020', '13,989.39489 IDR'),
 ('6/8/2020', '13,867.95377 IDR'),
 ('6/9/2020', '13,870.75718 IDR'),
 ('6/10/2020', '13,976.91611 IDR'),
 ('6/11/2020', '13,966.85142 IDR'),
 ('6/12/2020', '14,121.55442 IDR'),
 ('6/14/2020', '14,145.23543 IDR'),
 ('6/15/2020', '14,021.62200 IDR'),
 ('6/16/2020', '14,046.73959 IDR'),
 ('6/17/2020', '14,120.02241 IDR'),
 ('6/18/2020', '14,095.36729 IDR'),
 ('6/19/2020', '14,077.63950 IDR'),
 ('6/21/2020', '14,068.55627 IDR'),
 ('6/22/2020', '14,136.05128 IDR'),
 ('6/23/2020', '14,148.27391 IDR'),
 ('6/24/2020', '14,143.17320 IDR'),
 ('6/25/2020', '14,130.57047 IDR'),
 ('6/26/2020', '14,190.21303 IDR'),
 ('6/28/2020', '14,185.08225 IDR'),
 ('6/29/2020', '14,183.53912 IDR'),
 ('6/30/2020', '14,296.90560 IDR'),
 ('7/1/2020', '14,256.13040 IDR'),
 

## *Step 3 : Creating data frame & Data wrangling*

Create DataFrame using pandas

In [10]:
import pandas as pd
df = pd.DataFrame(temp, columns=('Date','Rate'))
df.head()

,Date,Rate
0,5/31/2020,"14,664.14584 IDR"
1,6/1/2020,"14,586.73900 IDR"
2,6/2/2020,"14,382.27395 IDR"
3,6/3/2020,"14,060.78946 IDR"
4,6/4/2020,"14,113.82802 IDR"


Check DataFrame

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    146 non-null    object
 1   Rate    146 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


Convert `Date` into Datetime64 dtype

In [12]:
df['Date']=df['Date'].astype('datetime64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    146 non-null    datetime64[ns]
 1   Rate    146 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 2.4+ KB


Convert *Rate* into float and removing `IDR` and `,`

In [13]:
df['Rate']=df['Rate'].str.replace(' IDR','')
df['Rate']=df['Rate'].str.replace(',','')

In [14]:
df['Rate']=df['Rate'].astype('float')

## *Step 4 : Visualizing and Conclusion*

I use plot graph to visualize daily rate IDR to USD Dollar

In [15]:
df.set_index('Date').plot()

In [16]:
df['Rate'].mean()

14514.333568219181

In [17]:
df.min()

Date    2020-05-31 00:00:00
Rate                  13868
dtype: object

In [18]:
df.max()

Date    2020-11-26 00:00:00
Rate                14892.3
dtype: object

After I finished visualize the data with plot graph, I'm using flash simple dashboard to show up the data.

In [19]:
#Using app.py and index.html in my file folder

Conclusion

Given a capstone to do a webscrapping on www.exchange-rates.org/history/IDR/USD/T to see IDR to Dollar USD rate in between 31th May 2020 and 26th November 2020. As we can see from the plot graph, IDR to US Dollar is very volatile. IDR Rate decreased in value from 14,664.14584 to 14,090.70769 which mean good for Indonesia. Where IDR Rate reach it high value on 28/09/2020 with 14,892.32 and reach the lowest value on 08/06/2020 with 13,867.95.